**Criando conexão no sistema.**

In [ ]:
from google.cloud.dataproc_spark_connect import DataprocSparkSession
from google.cloud.dataproc_v1 import Session
from pyspark.sql import functions as F
from pyspark.sql.functions import explode, col


# This will create a default Spark session
spark = DataprocSparkSession.builder.getOrCreate()


/usr/local/lib/python3.12/dist-packages/google/cloud/dataproc_spark_connect/session.py:332: UserWarning: Python version mismatch detected: Client is using Python 3.12, but Dataproc runtime 3.0 uses Python 3.11. This mismatch may cause issues with Python UDF (User Defined Function) compatibility. Consider using Python 3.11 for optimal UDF execution.
  dataproc_config: Session = self._get_dataproc_config()


██████████████████████████████████████████▏                                     

**Lendo tabelas da Silver**

In [ ]:
df_orders = spark.read.format("bigquery").option("table", "clinic-de.2_silver.orders").load()
df_orders_items = spark.read.format("bigquery").option("table", "clinic-de.2_silver.orders_items").load()


**Join para criação da tabela "items" e seleção de colunas**

In [ ]:
items = (
    df_orders_items.alias("oi")
    .join(df_orders.alias("o"), on="order_id", how="inner")
    .select(
        F.col("oi.order_item_id"),
        F.col("oi.order_id"),
        F.col("o.patient_id"),
        F.col("o.physician_id"),
        F.col("o.created_at").alias("ts_created"),
        "o.channel","o.priority","o.status","o.diagnosis",
        F.col("oi.test_code"), F.col("oi.test_name"),
        F.col("oi.price").alias("amount_item")
    )
)

**Lendo e criando tabela TL**


In [ ]:
df_samples_timeline = spark.read.format("bigquery").option("table", "clinic-de.2_silver.samples_timeline").load()
df_samples = spark.read.format("bigquery").option("table", "clinic-de.2_silver.samples").load()

**Join para criação da tabela "TL"**

In [ ]:
tl = (
    df_samples_timeline.alias("t")
    .join(df_samples.select("sample_id","order_item_id").alias("s"), on="sample_id", how="left")
    .groupBy("order_item_id")
    .agg(
        F.max(F.when(F.col("t.event")=="collected",  F.col("t.ts"))).alias("ts_collect"),
        F.max(F.when(F.col("t.event")=="received",   F.col("t.ts"))).alias("ts_received"),
        F.max(F.when(F.col("t.event")=="in_analysis",F.col("t.ts"))).alias("ts_analysis"),
        F.max(F.when(F.col("t.event")=="released",   F.col("t.ts"))).alias("ts_released"),
    )
)

**Lendo e criando tabela "Results"**

In [ ]:
df_results = spark.read.format("bigquery").option("table", "clinic-de.2_silver.results").load()

**Joins**


In [ ]:
f = (
    items.alias("items")
    .join(tl.alias("tl"), on="order_item_id", how="left")
    .join(df_results.alias("results"), on="order_item_id", how="left")
)

**Selecionando colunas**

In [ ]:
fato = (f.select(
    "order_item_id",
    "items.order_id",
    "items.patient_id",
    "items.test_code",
    "items.test_name",
    "physician_id",
    "ts_created",
    "channel",
    "priority",
    "status",
    "diagnosis",
    "amount_item",
    "ts_collect",
    "ts_received",
    "ts_analysis",
    "ts_released",
    "notes",
    "method",
    "value",
    "high",
    "low",
    "unit",
    "validated_by",
    "flag",
    "released_at",
    "result_id"
))

**Tabela "Fato criada"**

In [ ]:
fato.show()

  0%|           0/4 Tasks

+-------------+----------+----------+---------+-----------+------------+-------------------+---------------+--------+--------+---------+-----------+----------+-----------+-----------+-----------+-----+------+-----+----+----+----+------------+----+-----------+---------+
|order_item_id|  order_id|patient_id|test_code|  test_name|physician_id|         ts_created|        channel|priority|  status|diagnosis|amount_item|ts_collect|ts_received|ts_analysis|ts_released|notes|method|value|high| low|unit|validated_by|flag|released_at|result_id|
+-------------+----------+----------+---------+-----------+------------+-------------------+---------------+--------+--------+---------+-----------+----------+-----------+-----------+-----------+-----+------+-----+----+----+----+------------+----+-----------+---------+
| ITM-001392-1|ORD-001392|PAT-000710|      WBC| Leucócitos|  MED-000144|2025-10-05 23:53:58|         balcao| routine|canceled|    Z00.0|       18.0|      NULL|       NULL|       NULL|       

**Salvando tabela Fato na Gold**

In [ ]:
(items.write
        .format("bigquery")
        .option("table", "clinic-de.3_gold.tbl_fact_final")
        .option("writeMethod", "direct")
        .mode("overwrite")
        .save()
    )

print("Tabela salva com sucesso")

  0%|           0/2 Tasks

Tabela salva com sucesso


In [ ]:
spark.stop()